In [ ]:
import pandas as pd

# Đọc file Excel
def excel_to_conf(excel_file, conf_file):
    # Đọc dữ liệu từ file Excel (giả sử có 2 cột: key và value)
    df = pd.read_excel(excel_file)

    # Mở file .conf để ghi dữ liệu
    with open(conf_file, 'w') as conf:
        # Duyệt qua từng dòng của DataFrame
        for index, row in df.iterrows():
            key = row['Parameter']   # Cột key trong file Excel
            value = row['Setup Value']  # Cột value trong file Excel
            conf.write(f"{key}={value}\n")

# # Chạy hàm với file Excel và tên file .conf đầu ra
# excel_to_conf('sysctl.xlsx', 'new_sysctl.conf')


In [2]:
# import pandas as pd
# import os
# import tempfile

# def conf_to_excel(conf_file_path, excel_file_path):
#     # Đọc file .conf
#     data = []
#     with open(conf_file_path, 'r') as f:
#         for line in f:
#             line = line.strip()
#             if line and not line.startswith('#'):  # Bỏ qua dòng trống và comment
#                 parts = line.split('=', 1)  # Chia thành 2 phần theo dấu =
#                 if len(parts) == 2:
#                     key, value = parts
#                     data.append({
#                         'Parameter': key.strip(),
#                         'Setup Value': value.strip()
#                     })
    
#     # Tạo DataFrame
#     df = pd.DataFrame(data)
    
#     # Ghi ra file Excel
#     df.to_excel(excel_file_path, sheet_name='Config', index=False)
    
#     print(f"Đã chuyển đổi {conf_file_path} thành {excel_file_path}")
#     return excel_file_path

In [5]:
import configparser
import json
from deepdiff import DeepDiff

# Đọc file .conf và chuyển thành dictionary
def load_conf(file_path):
    config = configparser.ConfigParser()
    
    # Đọc nội dung file .conf
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Thêm một section giả (section 'default') nếu file không có section header
    if not content.startswith('['):
        content = '[default]\n' + content
    
    # Ghi nội dung có section header vào file tạm thời
    with open('temp.conf', 'w') as temp_file:
        temp_file.write(content)
    
    # Đọc file tạm thời và chuyển thành dictionary
    config.read('temp.conf')
    config_dict = {section: dict(config.items(section)) for section in config.sections()}
    
    return config_dict

# So sánh hai dictionary và trả về sự khác biệt dưới dạng JSON
def compare_conf(dict1, dict2):
    diff = DeepDiff(dict1, dict2, ignore_order=True)
    
    result = {}

    if not diff:
        result["message"] = "Same"
    else:
        result["message"] = "Different"
        
        # Nếu có sự thay đổi giá trị
        if 'values_changed' in diff:
            result["values_changed"] = []
            for key, change in diff['values_changed'].items():
                result["values_changed"].append({
                    "key": key,
                    "old_value": change['old_value'],
                    "new_value": change['new_value']
                })

        # Nếu có mục được thêm vào
        if 'dictionary_item_added' in diff:
            result["dictionary_item_added"] = []
            for key in diff['dictionary_item_added']:
                result["dictionary_item_added"].append({
                    "key": key,
                    # "added_in": "file 2"
                })

        # Nếu có mục bị xóa
        if 'dictionary_item_removed' in diff:
            result["dictionary_item_removed"] = []
            for key in diff['dictionary_item_removed']:
                result["dictionary_item_removed"].append({
                    "key": key,
                    # "removed_in": "file 2"
                })

    return result

# Đọc hai file .conf
file1 = '/workspaces/k8sconfig/edit_tom/sysctl.conf'
file2 = 'new_sysctl.conf'

data1 = load_conf(file1)
data2 = load_conf(file2)

# So sánh hai dictionary và lấy kết quả
diff_result = compare_conf(data1, data2)

# Lưu kết quả vào file JSON
with open('diff_result_conf.json', 'w') as json_file:
    json.dump(diff_result, json_file, indent=4)

print("File diff_result.json đã được lưu.")


File diff_result.json đã được lưu.


In [ ]:
# import os
# from deepdiff import DeepDiff

# def parse_conf_file(file_path):
#     config = {}
    
#     if not os.path.exists(file_path):
#         print(f"File không tồn tại: {file_path}")
#         return config
        
#     with open(file_path, 'r') as file:
#         for line in file:
#             line = line.strip()
#             # Bỏ qua các dòng trống hoặc dòng comment
#             if not line or line.startswith('#'):
#                 continue
                
#             # Phân tách key và value
#             if '=' in line:
#                 key, value = line.split('=', 1)
#                 config[key.strip()] = value.strip()
                
#     return config

# def compare_conf_files(file1_path, file2_path):
#     config1 = parse_conf_file(file1_path)
#     config2 = parse_conf_file(file2_path)
    
#     # Sử dụng deepdiff để tìm sự khác biệt
#     diff = DeepDiff(config1, config2)
    
#     if not diff:
#         print("Không có sự khác biệt giữa hai file.")
#         return
    
#     print("Các sự khác biệt:")
    
#     # Hiển thị các tham số bị thay đổi giá trị
#     if 'values_changed' in diff:
#         print("\nGiá trị thay đổi:")
#         for path, change in diff['values_changed'].items():
#             param = path.replace("root['", "").replace("']", "")
#             print(f"  {param}: {change['old_value']} -> {change['new_value']}")
    
#     # Hiển thị các tham số chỉ có trong file 1
#     if 'dictionary_item_removed' in diff:
#         print("\nTham số chỉ có trong file 1:")
#         for item in diff['dictionary_item_removed']:
#             param = item.replace("root['", "").replace("']", "")
#             value = config1[param]
#             print(f"  {param} = {value}")
    
#     # Hiển thị các tham số chỉ có trong file 2
#     if 'dictionary_item_added' in diff:
#         print("\nTham số chỉ có trong file 2:")
#         for item in diff['dictionary_item_added']:
#             param = item.replace("root['", "").replace("']", "")
#             value = config2[param]
#             print(f"  {param} = {value}")

In [ ]:
# conf_file = 'sysctl.conf'
# excel_file = 'a.xlsx'
excel_to_conf('sysctl.xlsx', 'new_sysctl.conf')
# conf_to_excel(conf_file, excel_file)
compare_conf(a,b)

Đã chuyển đổi sysctl.conf thành a.xlsx
Không có sự khác biệt giữa hai file.
